In [4]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import time

import warnings
warnings.filterwarnings('ignore')

In [197]:
def fetch_player_data(url):
    resp = requests.get(url)
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text.replace('<!--', '').replace('--!>', ''), 'html.parser')
    
    headers, rows = [], []
    
    table_data = soup.find_all('tbody')[2]
    table_rows = table_data.find_all('tr')
    
    for value in table_rows[0]:
        headers.append(value.get('data-stat'))
        
    for row in table_rows:
        row_values = []
        row_data = row.find_all('td')
        for data_value in row_data:
            if data_value.get('data-append-csv'):
                row_values.append(data_value.get('data-append-csv'))
            row_values.append(data_value.get_text())
        rows.append(row_values)
        
    df = pd.DataFrame(rows, columns = headers)
    
    df['minutes'] = df['minutes'].str.replace(',', '')
    
    df.rename(columns={'ranker' : 'player_id'}, inplace=True)
    df = df.apply(pd.to_numeric, errors = 'ignore')
    
    return df

In [198]:
def check_comp(comp):
    output = ""
    if comp.lower() == 'england' or comp.lower() == "premier_league" or comp.lower() == "eng":
        league_code = 9
        league_name = "Premier-League"
    elif comp.lower() == 'spain' or comp.lower() == "la_liga" or comp.lower() == "esp":
        league_code = 12
        league_name = "La-Liga"
    elif comp.lower() == 'italy' or comp.lower() == "serie-a" or comp.lower() == "ita":
        league_code = 11
        league_name = "Serie-A"
    elif comp.lower() == 'germany' or comp.lower() == "bundesliga" or comp.lower() == "ger":
        league_code = 20
        league_name = "Bundesliga"
    elif comp.lower() == 'france' or comp.lower() == "ligue-1" or comp.lower() == "fra":
        league_code = 13
        league_name = "Ligue-1"
    return league_code, league_name

In [199]:
def get_data(competition, year, data_type):
    
    comp = check_comp(competition)
    season = f"{str(year)}-{str(year+1)}"
    url = f"https://fbref.com/en/comps/{comp[0]}/{season}/{data_type}/{season}-{comp[1]}-Stats"
    df = fetch_player_data(url)
    
    return df

In [201]:
get_data("eng", 1998, "stats")

,player_id,player,nationality,position,team,age,birth_year,games,games_starts,minutes,...,pens_made,pens_att,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,matches
0,170ea618,Samassi Abou,fr FRA,FW,West Ham,25.0,1973.0,3.0,2.0,188.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches
1,26b5e727,Tony Adams,eng ENG,DF,Arsenal,31.0,1966.0,26.0,26.0,2340.0,...,0.0,0.0,4.0,0.0,0.04,0.00,0.04,0.04,0.04,Matches
2,39faf86a,Junior Agogo,gh GHA,FW,Sheffield Weds,18.0,1979.0,1.0,0.0,41.0,...,0.0,0.0,1.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches
3,8ddfb8b9,Gareth Ainsworth,eng ENG,MF,Wimbledon,25.0,1973.0,8.0,5.0,465.0,...,0.0,0.0,1.0,0.0,0.00,0.19,0.19,0.00,0.19,Matches
4,749d95dc,Philippe Albert,be BEL,DF,Newcastle Utd,30.0,1967.0,6.0,3.0,362.0,...,0.0,0.0,1.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,869d6ad3,Eddie Youds,eng ENG,DF,Charlton Ath,28.0,1970.0,22.0,21.0,1907.0,...,0.0,0.0,6.0,0.0,0.09,0.09,0.19,0.09,0.19,Matches
578,409a29d7,Luke Young,eng ENG,DF,Tottenham,19.0,1979.0,15.0,14.0,1279.0,...,0.0,0.0,5.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches
579,d3397c3e,Theodoros Zagorakis,gr GRE,MF,Leicester City,26.0,1971.0,20.0,16.0,1317.0,...,0.0,0.0,4.0,0.0,0.07,0.14,0.21,0.07,0.21,Matches
580,59883773,Gianfranco Zola,it ITA,FW,Chelsea,32.0,1966.0,37.0,35.0,2907.0,...,0.0,0.0,1.0,0.0,0.40,0.19,0.59,0.40,0.59,Matches


In [ ]:
## normal player stats work for all leagues and players

## if i try to got with teams it returns players sta
## if it try to go with 
